In [ ]:
import requests
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import xgboost as xgb
import joblib 
import numpy as np
import dagshub
import os
from dotenv import load_dotenv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [ ]:
DAGSHUB_TOKEN = os.getenv('DAGSHUB_TOKEN')
mlflow.set_tracking_uri(f'https://{DAGSHUB_TOKEN}@dagshub.com/Shurens/my-first-repo.mlflow')


In [ ]:
# API endpoint URLs
BASE_URL =  os.getenv('BASE_URL')  
TOKEN_URL = f"{BASE_URL}/token"
ALL_FILMS_URL = f"{BASE_URL}/all_films"


In [28]:
# Step 1: Authenticate and get the token
def get_token(username: str, password: str):
    response = requests.post(TOKEN_URL, data={"username": username, "password": password})
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception("Authentication failed, check your credentials")


In [29]:
# Step 2: Fetch all films data using the token
def fetch_all_films(token: str):
    headers = {
        "Authorization": f"Bearer {token}"
    }
    response = requests.get(ALL_FILMS_URL, headers=headers)
    
    if response.status_code == 200:
        films_data = response.json()["films"]
        return pd.DataFrame(films_data)
    else:
        raise Exception("Failed to fetch films data")


In [30]:
def train_model(df):
    # Étape 1: Préparation des données
    X = df.drop(columns=["f_evaluation"])  # Features except for the target
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(df["f_evaluation"])
    
    # Étape 2: Séparation des données en train et test avec stratification
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Vérification de la répartition des classes
    print("Répartition des classes dans l'ensemble d'entraînement :", 
          dict(zip(*np.unique(y_train, return_counts=True))))
    print("Répartition des classes dans l'ensemble de test :", 
          dict(zip(*np.unique(y_test, return_counts=True))))
    
    # Étape 3: Entraînement des modèles de classification
    
    # Modèle RandomForestClassifier
    with mlflow.start_run(run_name="RandomForestClassifier"):
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)
        y_pred_rf = rf_model.predict(X_test)
        accuracy_rf = accuracy_score(y_test, y_pred_rf)
        precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
        recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
        f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
        
        mlflow.log_metric("accuracy_rf", accuracy_rf)
        mlflow.log_metric("precision_rf", precision_rf)
        mlflow.log_metric("recall_rf", recall_rf)
        mlflow.log_metric("f1_rf", f1_rf)
        mlflow.log_artifact("label_encoder_films.joblib")
        mlflow.sklearn.log_model(rf_model, "random_forest_model", registered_model_name="random_forest_model")
    
    # Modèle XGBoostClassifier
    with mlflow.start_run(run_name="XGBoostClassifier"):
        xgb_model = xgb.XGBClassifier(objective="multi:softprob", n_estimators=100, random_state=42)
        xgb_model.fit(X_train, y_train)
        y_pred_xgb = xgb_model.predict(X_test)
        accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
        precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
        recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')
        f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
        
        mlflow.log_metric("accuracy_xgb", accuracy_xgb)
        mlflow.log_metric("precision_xgb", precision_xgb)
        mlflow.log_metric("recall_xgb", recall_xgb)
        mlflow.log_metric("f1_xgb", f1_xgb)
        mlflow.sklearn.log_model(xgb_model, "xgboost_model", registered_model_name="xgboost_model")
    
    # Modèle LogisticRegression
    with mlflow.start_run(run_name="LogisticRegression"):
        lr_model = LogisticRegression(max_iter=1000, random_state=42)
        lr_model.fit(X_train, y_train)
        y_pred_lr = lr_model.predict(X_test)
        accuracy_lr = accuracy_score(y_test, y_pred_lr)
        precision_lr = precision_score(y_test, y_pred_lr, average='weighted')
        recall_lr = recall_score(y_test, y_pred_lr, average='weighted')
        f1_lr = f1_score(y_test, y_pred_lr, average='weighted')
        joblib.dump(label_encoder, "label_encoder_films.joblib")
        
        mlflow.log_metric("accuracy_lr", accuracy_lr)
        mlflow.log_metric("precision_lr", precision_lr)
        mlflow.log_metric("recall_lr", recall_lr)
        mlflow.log_metric("f1_lr", f1_lr)
        mlflow.sklearn.log_model(lr_model, "logistic_regression_model", registered_model_name="logistic_regression_model")
    
    # Affichage des résultats
    print(f"RandomForestClassifier - Accuracy: {accuracy_rf}, Precision: {precision_rf}, Recall: {recall_rf}, F1-score: {f1_rf}")
    print(f"XGBoostClassifier - Accuracy: {accuracy_xgb}, Precision: {precision_xgb}, Recall: {recall_xgb}, F1-score: {f1_xgb}")
    print(f"LogisticRegression - Accuracy: {accuracy_lr}, Precision: {precision_lr}, Recall: {recall_lr}, F1-score: {f1_lr}")
    
    return {
        "RandomForestClassifier": {"accuracy": accuracy_rf, "precision": precision_rf, "recall": recall_rf, "f1": f1_rf},
        "XGBoostClassifier": {"accuracy": accuracy_xgb, "precision": precision_xgb, "recall": recall_xgb, "f1": f1_xgb},
        "LogisticRegression": {"accuracy": accuracy_lr, "precision": precision_lr, "recall": recall_lr, "f1": f1_lr}
    }

In [31]:
import os
import dotenv
username = 'shuren'
password = 'test'

# Step 1: Get the token
token = get_token(username, password)

# Step 2: Fetch films data
df = fetch_all_films(token)

# Step 3: Convert the data to a Pandas DataFrame
print(df.head())  # Preview the data to ensure it's loaded correctly

# Step 4: Train models on the data and log the results
results = train_model(df)
print("Model training complete. Results:")
print(results)


   f_budget  f_revenue  f_runtime  f_vote_count f_evaluation
0  25000000  124272124        146          1910         bien
1  45000000  788241776         89          5376         bien
2  64000000   97571250         92           756         bien
3  35000000  126216940        138          1310         bien
4  15000000   56255142        148          3045         bien
Répartition des classes dans l'ensemble d'entraînement : {np.int64(0): np.int64(151), np.int64(1): np.int64(152), np.int64(2): np.int64(152)}
Répartition des classes dans l'ensemble de test : {np.int64(0): np.int64(38), np.int64(1): np.int64(38), np.int64(2): np.int64(38)}


2024/11/03 17:16:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'random_forest_model' already exists. Creating a new version of this model...
2024/11/03 17:16:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model, version 3
Created version '3' of model 'random_forest_model'.
2024/11/03 17:16:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForestClassifier at: https://d0a87968a791705d505707f81cb9e16d8c269f9a@dagshub.com/Shurens/my-first-repo.mlflow/#/experiments/0/runs/36c90fa6113740db849a015d17c5d43c.
2024/11/03 17:16:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://d0a87968a791705d505707f81cb9e16d8c269f9a@dagshub.com/Shurens/my-first-repo.mlflow/#/experiments/0.
2024/11/03 17:16:23 WARNING mlflow.

RandomForestClassifier - Accuracy: 0.7456140350877193, Precision: 0.7447368421052633, Recall: 0.7456140350877193, F1-score: 0.7448176921861133
XGBoostClassifier - Accuracy: 0.7105263157894737, Precision: 0.715013794282087, Recall: 0.7105263157894737, F1-score: 0.7122317253962823
LogisticRegression - Accuracy: 0.543859649122807, Precision: 0.5190058479532164, Recall: 0.543859649122807, F1-score: 0.46940077466393254
Model training complete. Results:
{'RandomForestClassifier': {'accuracy': 0.7456140350877193, 'precision': np.float64(0.7447368421052633), 'recall': np.float64(0.7456140350877193), 'f1': np.float64(0.7448176921861133)}, 'XGBoostClassifier': {'accuracy': 0.7105263157894737, 'precision': np.float64(0.715013794282087), 'recall': np.float64(0.7105263157894737), 'f1': np.float64(0.7122317253962823)}, 'LogisticRegression': {'accuracy': 0.543859649122807, 'precision': np.float64(0.5190058479532164), 'recall': np.float64(0.543859649122807), 'f1': np.float64(0.46940077466393254)}}


In [ ]:
# import os
# import sys
# from mlflow.tracking import MlflowClient


# MLFLOW_URL = "http://localhost:5000"
# PROJECT_MODEL_NAME = "logistic_regression_model"
# MONITORER_DIR = "mlflow"
# experiment_name = "logistic_regression_model"

# def recuperer_nom_modeles(MLFLOW_URL, PROJECT_MODEL_NAME):
#     # Récupération des noms des modèles sur MLFLow
#     all_model_names=[]
#     model_registry_client = mlflow.tracking.MlflowClient(MLFLOW_URL)
#     model_versions = model_registry_client.search_model_versions("")
#     model_names = set([mv.name for mv in model_versions])
#     for name in model_names:
#         if PROJECT_MODEL_NAME in name:
#             all_model_names.append(name)
#     return all_model_names[0], model_registry_client

# def charger_modele_et_artefacts(model_name, model_registry_client, MONITORER_DIR):
#     """
#     Charge un modèle MLflow, ainsi que ses artefacts associés, et importe les modules Python associés.

#     Cette fonction récupère la version en production d'un modèle à partir du registre des modèles MLflow,
#     télécharge les artefacts associés, et importe dynamiquement tous les modules Python contenus dans les artefacts.
#     Les données de test et les prédictions associées au modèle sont également extraites.

#     Args:
#         model_name (str): Le nom du modèle à charger depuis le registre de modèles MLflow.
#         model_registry_client (mlflow.tracking.MlflowClient): Le client MLflow pour interagir avec le registre des modèles.
#         WORK_DIR (str): Le répertoire de travail où les artefacts du modèle seront téléchargés.

#     Returns:
#         tuple: Un tuple contenant deux éléments :
            
#     dict: Un dictionnaire avec les éléments suivants :
#     'model' (sklearn model): Le modèle MLflow chargé.
#     'run_id' (str): L'identifiant de la run associée au modèle.
#     'pickle' (str): Le chemin du fichier pickle des variables sauvegardées.
#     'X_test' (str): Les données de test utilisées lors de l'entraînement du modèle.
#     'y_test' (str): Les labels de test utilisés lors de l'entraînement du modèle.
#     'y_pred' (str): Les prédictions du modèle sur les données de test.
#     'rmse' (str or float): La valeur RMSE (Root Mean Square Error) du modèle, ou un message indiquant
#                             qu'aucune RMSE n'a été trouvée.
#     dict: Un dictionnaire des modules Python importés dynamiquement, avec les noms des modules comme clés.

#     Raises:
#         FileNotFoundError: Si le répertoire spécifié pour les artefacts n'existe pas ou n'est pas un répertoire.
#         ImportError: Si l'importation des modules Python échoue.

#     """
#     # Récupérer la version en production du modèle désigné
#     prod_model_version = model_registry_client.get_model_version_by_alias(model_name, "production")

#     # Charger le modèle MLflow et l'enregistrer dans un dictionnaire
#     model={}
#     model["model"] = mlflow.sklearn.load_model(prod_model_version.source)
#     model["run_id"] = prod_model_version.run_id

#     # Récupérer les artefacts du modèle
#     run = mlflow.get_run(model["run_id"])
#     artifact_uri = run.info.artifact_uri

#     # # Supprimer le dossier des artefacts s'il existe déjà
#     # if os.path.exists(MONITORER_DIR+"/model_artefacts/") and os.path.isdir(MONITORER_DIR+"/model_artefacts/"):
#     #     shutil.rmtree(MONITORER_DIR+"/model_artefacts/")

#     # Télécharger les artefacts du modèle
#     artifact_folder = mlflow.artifacts.download_artifacts(artifact_uri, dst_path=MONITORER_DIR+"/model_artefacts/")
#     sys.path.append(artifact_folder)

#     # Télécharger les artefacts du modèle
#     artifact_folder = mlflow.artifacts.download_artifacts(artifact_uri, dst_path=os.path.join(MONITORER_DIR, "model_artefacts"))
#     sys.path.append(artifact_folder)

#     # Charger le LabelEncoder
#     label_encoder_path = os.path.join(artifact_folder, "label_encoder_films.joblib")
#     if os.path.exists(label_encoder_path):
#         model["label_encoder"] = joblib.load(label_encoder_path)
#     else:
#         raise FileNotFoundError(f"Le fichier {label_encoder_path} n'a pas été trouvé.")


#     return model

# def tag_latest_model_as_production(MLFLOW_URL, experiment_name):
#     '''
#     Marque le modèle le plus récent comme étant le modèle en production.

#     Args:
#         mlflowURI (str): Adresse qui pointe vers MLFlow.
#         experiment_name (str): Nom du modèle dans le registre MLFlow.
#     '''
#     client = mlflow.tracking.MlflowClient(MLFLOW_URL)
#     registered_model = client.get_registered_model(experiment_name)

#     model_versions = client.search_model_versions(f"name='{experiment_name}'")
#     model_versions.sort(key=lambda x: x.last_updated_timestamp, reverse=True)

#     latest_model_version = model_versions[0] if model_versions else None
#     if latest_model_version:
#         client.set_registered_model_alias(registered_model.name, 'production', latest_model_version.version)

In [ ]:
# tag_latest_model_as_production(MLFLOW_URL, "logistic_regression_model")

In [ ]:
# # Appeler la fonction pour récupérer le nom du modèle et le client MLflow
# try:
#     model_name, client = recuperer_nom_modeles(MLFLOW_URL, PROJECT_MODEL_NAME)
#     print(f"Nom du modèle récupéré : {model_name}")
# except Exception as e:
#     print(f"Erreur lors de la récupération du nom du modèle : {e}")


Nom du modèle récupéré : logistic_regression_model


In [ ]:
# # Définir le répertoire de surveillance (assurez-vous que le chemin existe)
# if not os.path.exists(MONITORER_DIR):
#     os.makedirs(MONITORER_DIR)

# # Appeler la fonction pour charger le modèle et ses artefacts
# try:
#     model_data = charger_modele_et_artefacts(model_name, client, MONITORER_DIR)
#     label_encoder = model_data["label_encoder"]
#     loaded_model = model_data["model"]
#     run_id = model_data["run_id"]
#     print(f"Modèle chargé avec succès. Run ID : {run_id}")
# except Exception as e:
#     print(f"Erreur lors du chargement du modèle et des artefacts : {e}")


Erreur lors du chargement du modèle et des artefacts : Le fichier c:\Users\Utilisateur\Documents\projet_ia\bdd_films\mlflow\model_artefacts\artifacts\label_encoder.joblib n'a pas été trouvé.


In [ ]:
# encoder = joblib.load("label_encoder_films.joblib")
# model = mlflow.sklearn.load_model("mlflow/model_artefacts/artifacts/logistic_regression_model") 
# X_new = pd.DataFrame([[10550000, 15500000, 145, 75]], columns=["budget", "revenue", "runtime", "vote_count"])
# try:
#     prediction_numeric = loaded_model.predict(X_new)
#     prediction_label = label_encoder.inverse_transform(prediction_numeric.astype(int))
#     print(f"Prédiction pour les données {X_new.values} : {prediction_label[0]}")
# except Exception as e:
#     print(f"Erreur lors de la prédiction : {e}")


Prédiction pour les données [[10550000 15500000      145       75]] : moyen


In [ ]:
# import mlflow
# import joblib 
# import pandas as pd

# # Load the model
# logged_model = 'runs:/4c15cf97a8124539ad82139f53ba5fbe/random_forest_model'
# encoder = joblib.load("label_encoder_films.joblib")

# # Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Create a DataFrame for prediction input
# X_new = pd.DataFrame([[5500000, 141195658, 130, 562 ]], columns=["f_budget", "f_revenue", "f_runtime", "f_vote_count"])

# # Predict and assign the output to a variable
# prediction_numeric = loaded_model.predict(X_new)

# # Transform the numeric prediction to a label
# prediction_label = encoder.inverse_transform(prediction_numeric.astype(int))

# # Print the result
# print(f"Prédiction pour les données {X_new.values} : {prediction_label[0]}")


Prédiction pour les données [[  5500000 141195658       130       562]] : moyen


In [ ]:
# import pandas as pd
# import mlflow
# import joblib

# # Load the model and the encoder
# logged_model = 'runs:/4c15cf97a8124539ad82139f53ba5fbe/random_forest_model'
# encoder = joblib.load("label_encoder_films.joblib")
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Load the test data from CSV (assuming it's saved as 'test_data.csv')
# test_data = pd.read_csv('test_data.csv')

# # Prepare the features for prediction
# X_test = test_data[["f_budget", "f_revenue", "f_runtime", "f_vote_count"]]

# # Make predictions
# predictions_numeric = loaded_model.predict(X_test)

# # Convert numeric predictions to labels
# predictions_label = encoder.inverse_transform(predictions_numeric.astype(int))

# # Add the predictions to the DataFrame for comparison
# test_data['predicted_evaluation'] = predictions_label

# # Check how many predictions match the actual values
# correct_predictions = (test_data['predicted_evaluation'] == test_data['f_evaluation']).sum()

# # Validate the model with a threshold of 7 correct answers
# if correct_predictions >= 7:
#     print(f"Model validation successful: {correct_predictions} correct predictions.")
# else:
#     print(f"Model validation failed: {correct_predictions} correct predictions.")

# # Optionally, print the DataFrame to see the comparison
# print(test_data[['f_original_title', 'f_evaluation', 'predicted_evaluation']])


Model validation successful: 8 correct predictions.
                           f_original_title f_evaluation predicted_evaluation
0  From Dusk Till Dawn 2: Texas Blood Money     mediocre                 bien
1                                      Seed     mediocre             mediocre
2                             Spirited Away         bien                 bien
3           The Good, the Bad and the Ugly          bien                 bien
4                            Precious Cargo        moyen                moyen
5                                   Morbius        moyen                 bien
6                                Coriolanus        moyen                moyen
7                                    Volver        moyen                moyen
8                              The Runaways        moyen                moyen
9                               Thunderball        moyen                moyen
